# SPAH Postcode Mapping - Final merge & export

In [26]:
import pandas as pd
from datetime import datetime

### Import data

In [27]:
# define file pathways
cleaned_folder_path = r'C:\Users\scotth07\OneDrive - NHS Scotland\Documents\Networks & Projects\SPAH\Postcode Mapping\cleaned_data'
raw_folder_path = r'C:\Users\scotth07\OneDrive - NHS Scotland\Documents\Networks & Projects\SPAH\Postcode Mapping\raw_data'

patient_data_pathway = cleaned_folder_path + r'\patient_data.xlsx'
nearest_hospitals_pathway = cleaned_folder_path + r'\nearest_hospitals.xlsx' 

In [28]:
# import data
patient_df = pd.read_excel(patient_data_pathway, sheet_name='Sheet1')
nearest_hospitals_df = pd.read_excel(nearest_hospitals_pathway, sheet_name='Sheet1') 

### Merge using postcodes 

In [29]:
# merge the two dataframes to combine patient data and nearest hospital information
merged_df = pd.merge(patient_df, nearest_hospitals_df,
                    on='patient_postcode')                     

### Map health boards to different values consistent with the shape file values

In [30]:
health_board_mapping = {
    'NHS AYRSHIRE & ARRAN': 'A&A',
    'NHS BORDERS': 'BORD',
    'NHS FIFE': 'FIFE',
    'NHS FORTH VALLEY': 'FV',
    'NHS DUMFRIES & GALLOWAY': 'D&G',
    'NHS GRAMPIAN': 'GRAM',
    'NHS GREATER GLASGOW & CLYDE': 'GG&C', 
    'NHS HIGHLAND': 'HIGH',
    'NHS LANARKSHIRE': 'LAN',
    'NHS LOTHIAN': 'LOTH',
    'NHS ORKNEY': 'ORK',
    'NHS SHETLAND': 'SHET',
    'NHS TAYSIDE': 'TAY',
    'NHS WESTERN ISLES': 'WI'
    
}

In [32]:
merged_df['hb_residence_code'] = merged_df['hb_residence'].map(health_board_mapping)

### Define adult and paediatric patients

In [33]:
# assign todays date to variable
today = pd.to_datetime('today')

# assign date 16 years before today to variable
sixteen_years = today - pd.DateOffset(years=16)

In [34]:
# Classifies each patient as either adult or paediatric with the cut off age of 16
def adult_or_paediatric(dob):
    if pd.isna(dob):
        return None
    if dob >= sixteen_years:
        return 'Adult'
    return 'Paediatric'

In [35]:
# Apple the function to the dob column
merged_df['Adult/Paediatric'] = merged_df['dob'].apply(adult_or_paediatric)

### Export list of patients

In [37]:
merged_df.to_excel(cleaned_folder_path + r'\patient_hospitals_merged.xlsx',
                  index=False)